# Decomposição em valores singulares (SVD ou singular value decomposition)

<i>Créditos: grande parte deste notebook foi traduzido de https://github.com/fastai/numerical-linear-algebra</i>

## Aplicação: modelagem de tópicos

Uma aplicação importante de fatoração de matrizes é a modelagem de tópicos. Seja a **matriz termos por documentos**:

<img src="images/document_term.png" alt="term-document matrix" style="width: 80%"/>
(Source: [Introduction to Information Retrieval](http://player.slideplayer.com/15/4528582/#))

Podemos decompô-la em uma matriz alta e magra multiplicada por uma matriz curta e larga (possivelmente com uma matriz diagonal no meio).

Note que esta representação não leva em conta a ordem das palavras ou a estrutura da frase. Este é um exemplo de abordagem **bag of words**.

## Motivação

Considere o caso mais extremo: reconstruir a matriz usando um produto externo de dois vetores. Claramente, na maior parte dos casos não seremos capazes de reconstruir a matriz de maneira exata. Porém, usando um vetor com a frequência relativa de cada palavra e um com o número de palavras por documento, então o produto externo vai estar quase tão perto quanto possível.

Agora considere aumentar estas matrizes para duas colunas e duas linhas. A decomposição ótima agora seria agrupar (clusterizar) os documentos em dois grupos, cada um com uma distribuição de palavras tão diferente quanto possível da outra, mas tão similar quanto possível aos outros documentos no grupo. Nós iremos chamar estes dois grupos de "tópicos". Vamos agrupar as palavras em dois grupos, baseados naqueles que aparecem mais frequentemente em cada um dos tópicos.

## No restante da aula de hoje

Vamos pegar um dataset de documentos com categorias diferentes, encontrar os tópicos (que consistem em grupos de palavras) para eles. Neste caso, conhecer as categorias reais nos ajuda a avaliar se os tópicos encontrados fazem sentido.

Iremos fazer isso através do **Singular Value Decomposition (SVD)**.

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn import decomposition
from scipy import linalg
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
np.set_printoptions(suppress=True)

## Additional Resources
- [Data source](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html): Newsgroups are discussion groups on Usenet, which was popular in the 80s and 90s before the web really took off.  This dataset includes 18,000 newsgroups posts with 20 topics.
- [Chris Manning's book chapter](https://nlp.stanford.edu/IR-book/pdf/18lsi.pdf) on matrix factorization and LSI 
- Scikit learn [truncated SVD LSI details](http://scikit-learn.org/stable/modules/decomposition.html#lsa)

### Other Tutorials
- [Scikit-Learn: Out-of-core classification of text documents](http://scikit-learn.org/stable/auto_examples/applications/plot_out_of_core_classification.html): uses [Reuters-21578](https://archive.ics.uci.edu/ml/datasets/reuters-21578+text+categorization+collection) dataset (Reuters articles labeled with ~100 categories), HashingVectorizer
- [Text Analysis with Topic Models for the Humanities and Social Sciences](https://de.dariah.eu/tatom/index.html): uses [British and French Literature dataset](https://de.dariah.eu/tatom/datasets.html) of Jane Austen, Charlotte Bronte, Victor Hugo, and more

## Preparando os dados

Scikit Learn vem com vários datasets incluídos, além de ferramentas de carregamento para carregar diversos datasets externos padrão. Este é um [ótimo recurso](http://scikit-learn.org/stable/datasets/). Os Datasets incluem preços das moradias em Boston, imagens faciais, patches de floresta, diabetes, câncer de mama, e outros. Vamos usar o dataset de newsgroups.

Newsgroups são grupos de discussão criados no Usenet, que era uma rede popular nos anos 80 e 90 antes da web realmente estourar. Este dataset inclui 18000 posts em newsgroups espalhados em 20 tópicos.  

In [4]:
logging.basicConfig() # to show logging message of what is happening behind the scene.

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)

NameError: name 'logging' is not defined

A decomposição espectral, como o próprio nome diz, é aquela feita em função dos pares autovalores-autovetores de uma matriz quadrada. O SVD pode ser visto como uma generalização da decomposição espectral para matrizes não-quadradas. A ideia é encontrar a fatoração $A = U \Sigma V^\top$

In [ ]:
newsgroups_train.filenames.shape, newsgroups_train.target.shape

Vamos dar uma olhada nos dados. Você consegue advinhar em que categoria estão estas mensagens?

In [ ]:
print("\n".join(newsgroups_train.data[:3]))

In [ ]:
np.array(newsgroups_train.target_names)[newsgroups_train.target[:3]]

The target attribute is the integer index of the category.

Next, scikit learn has a method that will extract all the word counts for us.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups_train.data).todense() # (documents, vocab)
vectors.shape #, vectors.nnz / vectors.shape[0], row_means.shape

In [ ]:
print(len(newsgroups_train.data), vectors.shape)

In [ ]:
vocab = np.array(vectorizer.get_feature_names())

In [ ]:
vocab.shape

In [ ]:
vocab[7000:7020]